# MPI

MPI (Message Passing Interface) is a standard for writing parallel programs that can run on a distributed memory system. It is widely used in the HPC (High Performance Computing) community. There are implementations of MPI for many programming languages, including C, C++, Fortran, and Python. Much of the functionality and many of the commands we will see here will be similar in other languages. 

There are multiple implementations of MPI, including [OpenMPI](https://www.open-mpi.org/), [MPICH](https://www.mpich.org/), and [Intel MPI](https://www.intel.com/content/www/us/en/developer/tools/oneapi/mpi-library.html). If you are running this notebook in a Github Codespace, MPICH will already be installed. If you want to install MPICH on your local system you can follow the instructions [here](https://www.mpich.org/downloads/).

We will also be using the `mpi4py` package which is a Python wrapper for MPI. This is already installed if you'r running this notebook in a GitHUb Codespace, or you can install it locally using pip:

```bash
pip install mpi4py
```

Unlike the other methods we have seen so far, we run MPI from the terminal using the `mpiexec` command. We can run a Python script named `python_script.py` using MPI like this:

```bash
mpiexec -n 4 python python_script.py
```
In this command, the `-n 4` flag tells MPI to run the script using 4 processes. The section of the command `python python_script.py` tells which command MPI should be running on each process. This will run the script 4 times in parallel.

## Ranks

The script will be run once on each process and each copy of the code will be identified by its "rank". The rank is a unique integer identifier for each process which can be accessed using the `mpi4py.MPI.COMM_WORLD.Get_rank()` function. We can also get the number of ranks using the method `Get_rank()`. The code below shows how to access the rank:

```python
# Run this script with the terminal command `mpiexec -n 4 python get_rank.py`

import mpi4py.MPI as MPI

# Get a reference to the current MPI.COMM_WORLD communicator
comm = MPI.COMM_WORLD

# Get the total number of ranks in the communicator
n_rank = comm.Get_size()

# Get the rank of the current process
rank = comm.Get_rank()

# Print the rank of the current process
print(f'This script is being run by Rank {rank} out of {n_rank} total ranks')
```

This code can be found in the file [`04_mpi_scripts/get_rank.py`](04_mpi_scripts/get_rank.py). To run this command we will need to change directory in the terminal using the command:

```bash
cd 04_mpi_scripts
```

and run the script using MPI using the command:

```bash
mpiexec -n 4 python get_rank.py
```

## Communicating Between Ranks

In the above example, we access the variable `MPI.COMM_WORLD` which references a communicator. We can use a communicator to send messages between the processes it contains. The communicator `MPI.COMM_WORLD` contains all the processes that are running the script. It is possible to create other communicators which contain only a subset of the processes, which can be useful for more complex parallel programs, but we won't be covering that here.

We can send messages between ranks using the `send` and `recv` methods of the communicator. The code below shows how to send a message from rank 0 to rank 1:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # If we're in rank 0, send a dictionary to rank 1
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)
elif rank == 1:
    # If we're in rank 1, receive the dictionary from rank 0
    data = comm.recv(source=0)
    print(rank, data)
```

You can run this code from the file [04_mpi_scripts/send_recv.py](04_mpi_scripts/send_recv.py). The `send` method is used to send a dictionary from rank 0 to rank 1. By including it in the if-block, we make sure it is only called by rank 0. The first argument to `send` is the data to be sent. We also specify the destination rank so the message can be sent to rank 1. The `recv` method is called from rank 1 to receive the message from rank 0. The `source` argument specifies the rank of the process that sent the message. The data which is received is saved into the variable `data` and then printed. As we can see, we can send any type of data between ranks, including dictionaries, lists, and numpy arrays.

Both `send` and `recv` block, meaning that the program will wait at the `send` line until the message has been received by the destination rank, and will wait at the `recv` line until the message has been sent by the source rank. This means we need to plan carefully to make sure that the program doesn't get deadlocked. For example, the code below will deadlock as both ranks are waiting for the other to receive a message:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # Send a message to rank 1
    comm.send("Hello from rank 0", dest=1)
    # Receive the message from rank 1
    data = comm.recv(source=1)
elif rank == 1:
    # Send a message to rank 0
    comm.send("Hello from rank 1", dest=0)
    # Receive the message from rank 0
    data = comm.recv(source=0)
```

This code can be found in the file [`04_mpi_scripts/deadlock.py`](04_mpi_scripts/deadlock.py) and should be run with two processes.

## Non-Blocking Communication

To avoid deadlocks, we can use non-blocking communication. This allows the program to continue running while the message is being sent or received. This can be done using the `isend` and `irecv` functions. These returned a `Request` object. We can use the `wait` method of the `Request` object to wait until the message has been sent or received. If the `Request` object was created by `irecv` the `wait` method waits until it receives a value, then returns the value received. Using non-blocking communication can free up processes to do other work while the communication is pending. The code below shows how to use non-blocking communication:

```python
from mpi4py import MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # Send the integer 100 to rank 1
    req = comm.isend(100,dest=1)
    # Wait for the request to complete
    req.wait()
elif rank == 1:
    # Receive the integer from rank 0
    req = comm.irecv(source=0)
    # Wait for the request to complete and get the data
    data = req.wait()
    print(data)
```

This code can be found in the file [`04_mpi_scripts/non_blocking.py`](04_mpi_scripts/non_blocking.py) and should be run with two processes.

## Sending Numpy Arrays

The methods we've seen before like `send`, `recv`, `isend`, and `irecv` send Python objects between ranks using a process called [pickling](https://docs.python.org/3/library/pickle.html). This process allows an arbitrarily complex object to be serialized so they can be sent between ranks. Whilst flexible in terms of the types of object that can be sent, the process of pickling and unpicking adds a significant performance overhead to the sending of data between ranks.

However, some data types in Python, such as Numpy arrays, do not need to be pickled to be sent between ranks, which can speed up communication significantly. This can be done using functions with similar names to those we have already seen, except they begin with a capital letter, such as `Send` and `Recv`.

The syntax we have to use is a little different than before. We need to prepare an object to receive the data before we call the `Recv` method. This array should be a Numpy array with the same shape and data type as the array we are sending. This prepares a section of the memory in receiving rank to receive the data and is known as a buffer. The function `numpy.empty` is an efficient way to create this buffer. It will allocate the memory for th Numpy array but will not initialize it, meaning it will contain junk values. This is faster than using `numpy.zeros` which would initialize the array to zeros. For many Numpy functions, including `empty`, the `dtype` argument is used to [specify the data type](https://numpy.org/doc/2.1/reference/arrays.dtypes.html) of the array. There are a few ways to do this, but one ay is to use the Python type names such as `int`, `float`, `complex`, etc.

The code below shows how to send a Numpy array between ranks:

```python
from mpi4py import MPI
import numpy as np

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # If we're in rank 0, create an array of ten integers to send
    data = np.arange(10, dtype=float)
    # Send the array to rank 1
    comm.Send(data, dest=1)
elif rank == 1:
    # If we're in rank 1, create an array to receive the data
    data = np.empty(10, dtype=float)
    # data will initially contain junk values
    print('data before: ', data)
    # Receive the data from rank 0
    comm.Recv(data, source=0)
    print(rank, data)
```

This code can be found in the file [`04_mpi_scripts/numpy_send_recv.py`](04_mpi_scripts/numpy_send_recv.py) and should be run with two processes.

## Exercise: Sum of Powers of a Array

The function `random_float_array` in the file [`sum_of_powers.py`](04_mpi_scripts/sum_of_powers.py) generates a random array of integers. You should generate a random array of 100 floats with a minimum of zero and maximum of 10 using the function call `random_float_array(0, 10, 100)` on rank 0. Then send this array to all other ranks. Each rank, including rank 0, should calculate the value:

$$
\sum_{i=0}^{n-1} x_i^{r+1}
$$

where $x_i$ is the $i$ th element of the array and $r$ is the rank of the process. Each rank should then send the result back to rank 0. Rank 0 should assemble the results into a list `results` whose $i$ th element is the sum of the powers of the array elements calculated by rank $i$. Finally, rank 0 should print the list of results. This code should be able to be run with any number of ranks. For example, if the code is run on 2 ranks, you might receive the result:

```
[5079.372714885385, 34822.40206813637]
```

while on four ranks you might get

```
[5022.381368813411, 33480.47733076967, 251675.5452958118, 2018665.9149521096]
```

Note that you will get slightly different results as your array will contain different random numbers. The first entry is the sum of the array and is generated on rank 0, the next value is the sum of the square of the array and is calculated on rank 1, the next value is the sum of the cube of the array and is calculated on rank 2, and so on.

There is a sample solution in the file [`sample_solutions/sum_of_powers_solution.py`](sample_solutions/sum_of_powers.py).

## Collective Communication

In the last exercise, we send data from one rank to all other ranks. This sort of communication is a common thing you might want to do in parallel programs. MPI allows for collective communication which provides a convenient and efficient way to do this. The most common collective communication functions are `bcast`, `scatter`, `gather`, and `reduce`. These functions are called by all ranks in the communicator and communicates with all other ranks in the communicator. There are also equivalent functions that start with a capital letter, such as `Bcast`, `Scatter`, `Gather`, and `Reduce`, which you can use for sending Numpy arrays and other types of data that don't need to be pickled.

### Broadcast

The `bcast` function sends a single object from one rank to all other ranks in the communicator. The syntax is:

```python
import mpi4py.MPI as MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    # If the rank is 0, set the data to be broadcasted
    data = ['apples', 'bananas', 'cherries', 'dates']
else:
    # If the rank is not 0, we still need the variable to exist
    # Set it to None for now
    data = None

# Broadcast the data from rank 0 to all other ranks
data = comm.bcast(data, root=0)

# Each rank now has a copy of the data
print(f'Rank {rank} has data: {data}')
```

The code above can be found in the file [`04_mpi_scripts/broadcast.py`](04_mpi_scripts/broadcast.py) and should be run with 4 processes. This code is more compact and efficient that the equivalent code using `send` and `recv`.

### Scatter

The `scatter` function distributes the elements of an array from one rank to all other ranks in the communicator. The array should have the same number of entries as the number of ranks in the communicator. The syntax is:

```python
import mpi4py.MPI as MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
n_rank = comm.Get_size()

if rank == 0:
    # If the rank is 0, set the data to be scattered
    data = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    # Divide the data into equal parts
    n_per_rank = len(data) / n_rank
    data_local = []
    for i in range(n_rank):
        data_local.append(data[int(i*n_per_rank):int((i+1)*n_per_rank)])
else:
    # If the rank is not 0, we still need the variable to exist
    # Set it to None for now
    data_local = None

# Scatter the data from rank 0 to all other ranks
data_local = comm.scatter(data_local, root=0)

# Each rank now has a piece of the data in data_local
print(f'Rank {rank} has data: {data_local}')
```

The code above can be found in the file [`04_mpi_scripts/scatter.py`](04_mpi_scripts/scatter.py) and should be run with 4 processes.

One complication of the method that is reflected in the code above is that the data must be divided into as many parts as there are ranks. This can necessitate some preparation of the data as in the code above, which is a little cumbersome. However, the `scatter` function is more efficient than sending the data to each rank individually.

### Gather

The `gather` function is the opposite of `scatter`. It collects the data from all ranks in the communicator and assembles it into an array on one rank. The syntax is:

```python
import mpi4py.MPI as MPI

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
n_rank = comm.Get_size()

# We're aiming to write an array with square of each number.
# We'll compute part of the array on each rank and then gather the results.
n = 10
i_local_start = int(rank * n / n_rank)
i_local_stop = int((rank + 1) * n / n_rank)
data_local = [i**2 for i in range(i_local_start, i_local_stop)]

# Print the local data on each rank
print(f'Rank {rank} has data: {data_local}')

# Gather the data from all ranks to rank 0
data = comm.gather(data_local, root=0)

# Rank 0 now has all the data
if rank == 0:
    print(f'Rank {rank} has data: {data} before flattening')
    # Flatten the list of lists into a single list
    data = sum(data, [])
    print(f'Rank {rank} has data: {data} after flattening')
```

The code above can be found in the file [`04_mpi_scripts/gather.py`](04_mpi_scripts/gather.py) and should be run with 4 processes. Again, we've had to do a little work to work out which parts of the data each rank should be working on in advance.

### Reduce

The `reduce` function is used to combine data from all ranks in the communicator into a single value. The syntax is:

```python
# Run this script with the terminal command `mpiexec -n 4 python gather.py`

import mpi4py.MPI as MPI
import numpy as np

# Get the communicator and the rank of the process
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
n_rank = comm.Get_size()

# We're going to generate 10 random numbers between 0 and 1 and count how many are less than 0.5
n = 10

# Calculate how many numbers each rank will generate
n_local = int(n / n_rank)
if rank < n % n_rank:
    # If the rank is less than the remainder, it will generate one more number
    # This is to ensure that the right number of numbers are generated
    n_local += 1

# Generate the random numbers
numbers_local = np.random.rand(n_local)
count_local = np.sum(numbers_local < 0.5)

# Print the local data on each rank
print(f'Rank {rank} has generated {n_local} numbers, and {count_local} are less than 0.5')

# Reduce the count of numbers less than 0.5 from all ranks to rank 0
count = comm.reduce(count_local, op=MPI.SUM, root=0)

# Rank 0 now has the total count
if rank == 0:
    print(f'Rank {rank} has the overall result of {count} numbers less than 0.5')
else:
    # Reduce returns None on all ranks other than the root
    print(f'On rank {rank}, the value of count is {count}')

# If we want all ranks to have the total count, we can use allreduce
# This does not require a root rank
count_all = comm.allreduce(count_local, op=MPI.SUM)

print(f'On rank {rank}, the value of count_all is {count_all}')
```

The code above can be found in the file [`04_mpi_scripts/reduce.py`](04_mpi_scripts/reduce.py) and should be run with 4 processes. There are a couple of things to note in the code above. The first is that the `reduce` function only returns a value on the root rank. On all other ranks, it returns `None`. If we want each rank to have a copy of the result, we can use the `allreduce` function. The second thing to note is that we have to specify an operation to the `op` argument to perform on the data to combine it into a single value. In this case, we are using `MPI.SUM` to add the counts from each rank together. There are other operations available, some common ones are:

* `MPI.SUM` - Adds the values together
* `MPI.PROD` - Multiplies the values together
* `MPI.MAX` - Returns the maximum value
* `MPI.MIN` - Returns the minimum value

## Exercise: Share Price Prediction

Share prices change over time in an unpredictable way. However, we can simulate how the share price might change over time using a geometric random walk. We can say that the price of a share on a given day is equal to the price of the share on the previous day multiplied by a random number drawn from a normal distribution with a given mean and standard deviation. We can simulate this process by generating a random number for each day and multiplying it by the share price on the previous day. We can do this for a number of days to simulate the share price over time. In the file [`share_price.py`](04_mpi_scripts/share_price.py) there is a function `simulate_share_price` which takes the initial share price, the mean of the normal distribution, the standard deviation of the normal distribution, and the number of days to simulate. This function returns the share price at the end of the final day of the simulated period.

Each call to this function will return a different final share price as a different set of random numbers will have been generated. This means we can call the function multiple times to get a distribution of final share prices. Your task is to write code which will call this code `n` times in total, split across each rank in the communicator. Once this is done, use collective communication to collect the results on rank zero and print the mean, standard deviation, minimum and maximum values of the final share prices. The code should be able to be run with any number of ranks. Your code should have values for the initial share price, mean, standard deviation, and number of days to simulate, and the number of simulations to run hard-coded into the script. Try the following values:

* Initial share price: 100
* Mean fractional daily change: 0.001
* Standard deviation of fractional daily change: 0.02
* Number of days to simulate: 100
* Number of simulations: 1000

As a reminder, the equations are the mean and standard deviation are:

$$
\bar{x} = \frac{1}{n} \sum_{i=0}^{n-1} x_i\\
\sigma = \sqrt{\frac{1}{n} \sum_{i=0}^{n-1} (x_i - \bar{x})^2} = \sqrt{\frac{1}{n} \sum_{i=0}^{n-1} x_i^2 - \bar{x}^2}
$$

where $\bar{x}$ is the mean, $\sigma$ is the standard deviation and $x_i$ is the value of each piece of data (the final share price of the $i$ th simulation in this case). There are two forms of the standard deviation equation given, you may use whichever you think is more appropriate.

Think carefully about how to structure your code and use the collective communication functions you've seen. There is a sample solution in the file [`sample_solutions/share_price.py`](sample_solutions/share_price.py). For reference, when I ran the sample solution, I got the following output:

* Mean: 110.70897767200135
* Standard deviation: 23.170420016985695
* Minimum: 54.494856062641915
* Maximum: 220.93130434066072

Your code will probably not produce the same results as the random numbers generated will be different, but your results should be similar.